In [18]:
import boto3
import os,sys
import datetime as dt
import glob
import pandas as pd
pd.set_option('display.max_rows', 1000)
import hvplot.pandas
import holoviews as hv
import subprocess as sp

In [19]:
os.makedirs('/dev/shm/logs',exist_ok=True)
os.chdir('/dev/shm/logs')

In [49]:
!aws s3 sync s3://earthbigdata/logs/ /dev/shm/logs --quiet

In [50]:
def make_entry(ip_dict,date_dict,logfile):  
    with open(logfile,"r") as f:
        lines=f.readlines()
    date = lines[0].split('[')[1].split('+')[0].strip()
    date = dt.datetime.strptime(date,'%d/%b/%Y:%H:%M:%S')
    ipaddr=lines[0].split(']')[1].split('-')[0].strip()
    if not ipaddr in ip_dict:
        ip_dict[ipaddr]=[]
    if not date.date() in date_dict:
        date_dict[date.date()]=0
    ip_dict[ipaddr].append(date)
    date_dict[date.date()]+=1
    

In [51]:
ip_dict={}
date_dict={}

for log in glob.glob('ebd-covid19*'):
    make_entry(ip_dict,date_dict,log)

In [52]:
access=pd.DataFrame.from_dict(date_dict,orient='index')

In [53]:
access.sort_index(inplace=True,ascending=False)

In [54]:
access.hvplot.barh(padding=0.1,grid=True,ylabel='Access Times per day',xlabel='',title='EBD COVID 19 Access')

:Bars   [index]   (0)

In [9]:
access.head()

,0
2020-04-09,109
2020-04-08,292
2020-04-07,257
2020-04-06,317
2020-04-05,350


In [ ]:
ip_count={}
for i in ip_dict:
    ip_count[i]=len(ip_dict[i])

In [ ]:
access_count=pd.DataFrame.from_dict(ip_count,orient='index')

access_count=access_count[access_count>=5].dropna()

access_count = access_count.sort_values(0,ascending=False)

access_count['nslookup']=''
for i in access_count.index:
    cmd='nslookup {}'.format(i)
    try:
        ret= sp.check_output(cmd.split())
        nslookup = ret.decode().split('name = ')[1].split('\n')[0].strip('.')
        # print(nslookup)
        access_count.loc[i,'nslookup']=nslookup
    except:
        pass

In [ ]:
access_count.head(40)

In [ ]:
len(list(ip_dict.keys()))

In [ ]:
list(ip_dict.keys())[:10]

In [ ]:
for i in list(ip_dict.keys()):
    if ip_dict[i][-1].date() == dt.datetime.now().date():
        if i in list(access_count.index):
            print(ip_dict[i][-1].date(),access_count.loc[i][0],'\t',access_count.loc[i].nslookup)

In [ ]:
access_list=[]
for i in ip_dict:
    last_access=ip_dict[i][-1].date()
    if i in list(access_count.index):
        a = int(access_count.loc[i][0])
        who = access_count.loc[i].nslookup
        access_list.append((last_access,a,i,who))
access_list.sort(reverse=True)
for i in access_list:
    print('{} {:5d}  {:17s} {}'.format(*i))